# Treinamento com interface de alto nível

## Importação das bibliotecas

In [1]:
# http://pytorch.org/
from os.path import exists

import torch

In [2]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

## Criação da rede

In [17]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = nn.Linear(784, 1568)
        self.layer2 = nn.Linear(1568, 3136)
        self.layer3 = nn.Linear(3136, 1568)
        self.layer3_4 = nn.Linear(1568, 1568)
        self.layer4 = nn.Linear(1568, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = self.layer1(x)
        x = F.relu(x)
        x = self.layer2(x)
        x = F.relu(x)
        x = self.layer3(x)
        x = F.relu(x)
        x = self.layer4(x)
        x = F.relu(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

model = Net()

In [15]:
model

Net(
  (layer1): Linear(in_features=784, out_features=1568, bias=True)
  (layer2): Linear(in_features=1568, out_features=3136, bias=True)
  (layer3): Linear(in_features=3136, out_features=1568, bias=True)
  (layer4): Linear(in_features=1568, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

## Treinamento

### Criando o objeto de treinamento

In [4]:
def train(log_interval, dry_run, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if dry_run:
                break

In [5]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

## Avaliação

In [6]:
use_cuda = torch.cuda.is_available()
use_cuda

True

In [22]:
use_cuda = torch.cuda.is_available()

torch.manual_seed(1111)

device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {'batch_size': 64}
test_kwargs = {'batch_size': 1000}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                    'pin_memory': True,
                    'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                    transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                    transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=1)

epochs = 8
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

for epoch in range(1, epochs + 1):
    train(10, False, model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.329428

Test set: Average loss: 2.2858, Accuracy: 1504/10000 (15%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 2.285605

Test set: Average loss: 2.2654, Accuracy: 2251/10000 (23%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 2.275880

Test set: Average loss: 2.2471, Accuracy: 3398/10000 (34%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 2.261122

Test set: Average loss: 2.2292, Accuracy: 4873/10000 (49%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 2.240219

Test set: Average loss: 2.2116, Accuracy: 6091/10000 (61%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 2.213489

Test set: Average loss: 2.1998, Accuracy: 4555/10000 (46%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 2.203644

Test set: Average loss: 2.1906, Accuracy: 5560/10000 (56%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 2.206129


KeyboardInterrupt: 

In [19]:
aux = torch.load("mnist_cnn.pt")

<ipython-input-19-cb91ad9ab067>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  aux = torch.load("mnist_cnn.pt")


In [20]:
aux

OrderedDict([('layer1.weight',
              tensor([[-0.0003, -0.0271,  0.0240,  ...,  0.0353,  0.0151, -0.0153],
                      [-0.0208, -0.0123,  0.0298,  ...,  0.0074, -0.0148, -0.0048],
                      [ 0.0010, -0.0267, -0.0105,  ...,  0.0171,  0.0191,  0.0073],
                      ...,
                      [-0.0194,  0.0303,  0.0269,  ...,  0.0003,  0.0118, -0.0055],
                      [ 0.0055, -0.0117, -0.0062,  ..., -0.0226, -0.0015, -0.0182],
                      [ 0.0190, -0.0259, -0.0127,  ..., -0.0030,  0.0031, -0.0264]],
                     device='cuda:0')),
             ('layer1.bias',
              tensor([-0.0177, -0.0261, -0.0074,  ..., -0.0339, -0.0301,  0.0227],
                     device='cuda:0')),
             ('layer2.weight',
              tensor([[-0.0271, -0.0070, -0.0012,  ..., -0.0216,  0.0065, -0.0299],
                      [-0.0017,  0.0243,  0.0073,  ...,  0.0054, -0.0276,  0.0368],
                      [-0.0211, -0.0065,  0.00

In [21]:
model.load_state_dict(aux)

<All keys matched successfully>